In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from os import listdir
import statsmodels.api as sm

/home/harshit/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [90]:
#path = "/home/harshit/Project/Returns/2017/"
#entropy=pd.read_csv(path+"continuous_entropy.csv")

In [3]:
path = "/home/harshit/Project/2017/"
entropy=pd.read_csv(path+"entropy.csv")

In [4]:
entropy["ent_risk"] = np.exp(entropy["Entropy"])

In [5]:
data_path = "/home/harshit/Project/2017/"              # finding annual returns of all companies
lis = pd.DataFrame(listdir(data_path),columns = ["Symbol"])
df = pd.DataFrame(columns=["Symbol","ann_return"])

for i in (lis["Symbol"]):            
    if i == "renyi_entropy.csv": continue
    if i == "entropy.csv": continue
    data = pd.read_csv(data_path+i)
    name = i.split('.')[0]
    last = data.iloc[-1][name]
    first= data.iloc[0][name]
    ann_ret = (last-first)*100/first
    df2=pd.DataFrame([[name, ann_ret]],columns=["Symbol","ann_return"])
    df = df.append(df2,ignore_index=True) # a frame having annual returns of all firms

combined = pd.merge(df,entropy,on='Symbol')

In [28]:
#Using co-variance to find beta_risk measure and finding standard deviation of annual returns of firms
factors = pd.read_csv("/home/harshit/Project/Returns/averages/FourFactors.csv")

eq_path = "/home/harshit/Project/Returns/2017/"
lis = pd.DataFrame(listdir(eq_path),columns = ["Symbol"])    
df = pd.DataFrame(columns=["Symbol","beta_risk"])
df4 = pd.DataFrame(columns=["Symbol","std_risk"])
for i in (lis["Symbol"]):
    if i == "continuous_renyi_entropy.csv": continue
    if i == "continuous_entropy.csv": continue
    equity = pd.read_csv(eq_path+i)
    name = i.split(".")[0]
    equity=equity.drop([0])
    equity = pd.merge(factors,equity,on="Date")
    equity["R-Rf %"] = equity["Return"] - equity["Rf %"]
#beta risk = cov(rm-rf,r-rf)/var(rm-rf)

    cov = np.sum((equity["R-Rf %"]-np.mean(equity["R-Rf %"]))*((equity["Rm-Rf %"]-np.mean(equity["Rm-Rf %"]))))/len(equity)
    var = np.sum((equity["Rm-Rf %"]-np.mean(equity["Rm-Rf %"]))**2)/len(equity)
    beta_risk = cov/var
    std = np.std(equity["R-Rf %"])
    df2 = pd.DataFrame([[name,beta_risk]],columns=["Symbol","beta_risk"])
    df = df.append(df2,ignore_index=True)
    df3 = pd.DataFrame([[name,std]],columns=["Symbol","std_risk"])
    df4 = df4.append(df3,ignore_index=True)

data = pd.merge(combined,df,on="Symbol")
all_data= pd.merge(data,df4,on="Symbol")

In [14]:
X1 = all_data[["beta_risk"]]
X1 = sm.add_constant(X1)
y = all_data[["ann_return"]]
model1 = sm.OLS(y,X1,hasconst=True).fit()

In [10]:
X2 = all_data[["ent_risk"]]
X2 = sm.add_constant(X2)
y = all_data[["ann_return"]]
model2 = sm.OLS(y,X2,hasconst=True).fit()

In [30]:
X1 = all_data[["std_risk"]]
X1 = sm.add_constant(X1)
y = all_data[["ann_return"]]
model3 = sm.OLS(y,X1,hasconst=True).fit()

In [31]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ann_return   R-squared:                       0.064
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     76.71
Date:                Thu, 22 Nov 2018   Prob (F-statistic):           7.10e-18
Time:                        07:49:07   Log-Likelihood:                -6683.2
No. Observations:                1133   AIC:                         1.337e+04
Df Residuals:                    1131   BIC:                         1.338e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.4116      6.337      0.854      0.393      -7.021      17.844
beta_risk    -40.4189      4.615     -8.759      0.000     -49.473     -31.364
==============================================================================
Omnibus:                     1287.079   Durbin-Watson:                   2.090
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           186979.096
Skew:                           5.398   Prob(JB):                         0.00
Kurtosis:                      65.001   Cond. No.                         4.87
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ann_return   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     19.20
Date:                Thu, 22 Nov 2018   Prob (F-statistic):           1.29e-05
Time:                        07:49:15   Log-Likelihood:                -6710.8
No. Observations:                1133   AIC:                         1.343e+04
Df Residuals:                    1131   BIC:                         1.344e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         47.1851      3.348     14.093      0.000      40.616      53.754
ent_risk       0.5037      0.115      4.381      0.000       0.278       0.729
==============================================================================
Omnibus:                     1262.014   Durbin-Watson:                   2.086
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           166468.991
Skew:                           5.246   Prob(JB):                         0.00
Kurtosis:                      61.448   Cond. No.                         36.3
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             ann_return   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     2.800
Date:                Thu, 22 Nov 2018   Prob (F-statistic):             0.0946
Time:                        07:49:28   Log-Likelihood:                -6718.9
No. Observations:                1133   AIC:                         1.344e+04
Df Residuals:                    1131   BIC:                         1.345e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         53.0247      3.217     16.482      0.000      46.713      59.337
std_risk       1.0881      0.650      1.673      0.095      -0.188       2.364
==============================================================================
Omnibus:                     1273.834   Durbin-Watson:                   2.092
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           172796.182
Skew:                           5.325   Prob(JB):                         0.00
Kurtosis:                      62.556   Cond. No.                         5.95
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
len(all_data)

1133